In [14]:
import json, codecs, csv

In [15]:
# Open the CSV containing the content.
infile = codecs.open('../data/dll-works-export.csv','r',encoding='utf-8')
work_data = csv.reader(infile,delimiter=',')

# Create an iterable for the script.
works = [row for row in work_data]

# Remove the header row.
del works[0]

In [16]:
data = {}
data['@context']={}
data['@context']['@base'] = 'https://catalog.digitallatin.org/'
data['@context']['madsrdf'] = 'http://www.loc.gov/mads/rdf/v1#'
data['@context']['rdf'] = 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'
data['@context']['dcterms'] = 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'
data['@context']['Title'] = 'madsrdf:Title'
data['@context']['Variant'] = 'madsrdf:hasVariant'
data['@context']['VariantTitle'] = 'madsrdf:variantLabel'
data['@context']['WorkAuthority'] = 'madsrdf:isMemberOfMADSCollection'
data['@context']['Abbreviation'] = 'madsrdf:hasAbbreviationVariant'
data['@context']['HasPart'] = 'dcterms:hasPart'
data['@context']['PartOf'] = 'dcterms:isPartOf'
data['@context']['References'] = 'dcterms:references'
data['@context']['TextGroup'] = 'madsrdf:isMemberOfMADSCollection'
data['@context']['Author'] = 'madsrdf:Authority'
data['@context']['AuthorName'] = {}
data['@context']['AuthorName']['@id'] = 'madsrdf:PersonalName'
data['@context']['AuthorName']['@type'] = '@id'
data['@context']['AuthorizedName'] = 'madsrdf:authoritativeLabel'
data['@context']['DubiousAttribution'] = {}
data['@context']['DubiousAttribution']['@id'] = 'dcterms:description'
data['@context']['DubiousAttribution']['@type'] = '@id'
data['@context']['AttributedName'] = {}
data['@context']['AttributedName']['@id'] = 'madsrdf:PersonalName'
data['@context']['AttributedName']['@type'] = '@id'
data['@context']['AttributedAuthorName'] = 'madsrdf:Name'
data['@context']['Dubious'] = 'madsrdf:hasChararacteristic'
data['@context']['Identifier'] = {}
data['@context']['Identifier']['@id'] = 'madsrdf:Identifier'
data['@context']['Identifier']['@type'] = '@id'
data['@context']['DLLid'] = 'madsrdf:idValue'
data['@context']['STOAid'] = 'madsrdf:idValue'
data['@context']['PHIid'] = 'madsrdf:idValue'
data['@context']['CTS-ID'] = 'madsrdf:idValue'
data['@context']['CTS-URN'] = 'madsrdf:idValue'
data['@graph'] = []

for i in works:
    dll_link = i[0]
    title = i[1]    
    
    # Handle the alternative title field: test for multiple values; return appropriate format.
    def alt_title_process():
        alternative_title = i[2]
        # If there aren't any values, return an empty value.
        if not alternative_title:
            alternative_title = ''
            return alternative_title

        # If there is > 1 value, the values will be separated by ';'. Process the
        # values as a list, then return the joined list.
        elif ';' in alternative_title:
            alternative_titles = []
            alt_title_list = alternative_title.split(';')
            for title in alt_title_list:
                alternative_titles.append(title)
            alternative_title = alternative_titles
            return alternative_title

        # If there is only one alternative title, return it.
        else:
            return alternative_title

    alt_title = alt_title_process()
    
    work_authority = i[3]
    abbreviation = i[4]
    
    # Handle value(s) in field 'Has Part'. There might be > 1.
    def has_part_process():
        has_part = i[5]
        # If there aren't any values, return an empty value.
        if not has_part:
            has_part = ''
            return has_part

        # If there is > 1 value, the values will be separated by ';'. Process the
        # values as a list, then return the joined list.
        elif ';' in has_part:
            has_parts = []
            has_parts_list = has_part.split(';')
            for work in has_parts_list:
                has_parts.append(work)
            has_part = has_parts
            return has_part

        # If there is only one value, return it.
        else:
            return has_part

    has_part = has_part_process()
    
    # Handle value(s) in field 'Part of'. There might be > 1.
    def part_of_process():
        part_of = i[6]
        # If there aren't any values, return an empty value.
        if not part_of:
            part_of = ''
            return part_of

        # If there is > 1 value, the values will be separated by ';'. Process the
        # values as a list, then return the joined list.
        elif ';' in part_of:
            part_ofs = []
            part_ofs_list = part_of.split(';')
            for work in part_ofs_list:
                part_ofs.append(work)
            part_of = part_ofs
            return part_of

        # If there is only one value, return it.
        else:
            return part_of

    part_of = part_of_process()
    
    # Handle value(s) in field 'References'. There might be > 1.
    def references_process():
        references = i[7]
        # If there aren't any values, return an empty value.
        if not references:
            references = ''
            return references

        # If there is > 1 value, the values will be separated by ';'. Process the
        # values as a list, then return the joined list.
        elif ';' in references:
            references_many = []
            has_reference_list = references.split(';')
            for work in has_references_list:
                references_many.append(work)
            references = references_many
            return references

        # If there is only one value, return it.
        else:
            return references

    references = references_process()
    
    # Handle value(s) in field 'Authors'. There might be > 1.
    def author_process():
        author = i[8]
        # If there aren't any values, return an empty value.
        if not author:
            author = ''
            return author

        # If there is > 1 value, the values will be separated by ';'. Process the
        # values as a list, then return the joined list.
        elif ';' in author:
            authors = []
            author_list = author.split(';')
            for author in author_list:
                authors.append(author)
            author = authors
            return author

        # If there is only one value, return it.
        else:
            return author
        
    dll_author = author_process()
    
    # Handle value(s) in field 'Attribution'. There might be > 1.
    def attribution_process():
        attribution = i[9]
        # If there aren't any values, return an empty value.
        if not attribution:
            attribution = ''
            return attribution

        # If there is > 1 value, the values will be separated by ';'. Process the
        # values as a list, then return the joined list.
        elif ';' in attribution:
            attributions = []
            attribution_list = attribution.split(';')
            for attribution in attribution_list:
                attributions.append(attribution)
            attribution = attributions
            return attribution

        # If there is only one value, return it.
        else:
            return attribution
        
    attribution = attribution_process()
    
    
    dubious = i[10]
    dll_work_id = i[11]
    phi_number = i[12]
    stoa_number = i[13]
    cts_urn = i[14]
    
    work = {'@id': dll_link,
           'Title': title,
           'Variant': alt_title,
           'WorkAuthority': work_authority,
           'Abbreviation': abbreviation,
           'HasPart': has_part,
           'PartOf': part_of,
           'References': references,
            'Author': {'AuthorName': {'AuthorizedName': dll_author}},
            'DubiousAttribution': {'Dubious': dubious,
                            'AttributedName':{'AttributedAuthorName': attribution}},
            'Identifier': {'ExternalID': external_identifier,
                          'PHIid': phi_number,
                          'STOAid': stoa_number,
                          'CTS-URN': cts_urn},
            }
                            
    data['@graph'].append(work)   

In [17]:
with codecs.open('../results/works.json','a',encoding='utf-8') as outfile:
    json.dump(data,outfile,sort_keys = True, indent = 4, ensure_ascii = False)